In [1]:
import numpy as np
import pennylane as qml 
import torch

def create_circuit(n_qubits,n_layers=None,circ = "simplified_two_design",fim=False, shots=None):

    dev = qml.device("default.qubit.torch", wires=n_qubits, shots=shots)

    def RZRY(params):
        #qml.SpecialUnitary(params, wires=range(n_qubits))
        #qml.SimplifiedTwoDesign(initial_layer_weights=init_params, weights=params, wires=range(n_qubits))
        #qml.AngleEmbedding(params,wires=range(n_qubits))
        for q in range(n_qubits):
            qml.Hadamard(wires=q)

        for w in range(n_layers): 
            for q in range(n_qubits):
                index = w * (2*n_qubits) + q * 2
                qml.RZ(params[index],wires=q)
                qml.RY(params[index + 1],wires=q)
        
        qml.broadcast(qml.CNOT , wires=range(n_qubits), pattern="all_to_all")
        
        return qml.probs(wires=range(n_qubits))

    def S2D(init_params,params,measurement_qubits=0,prod_approx=False):
        #qml.SpecialUnitary(params, wires=range(n_qubits))
        qml.SimplifiedTwoDesign(initial_layer_weights=init_params, weights=params, wires=range(n_qubits))
        
        #qml.broadcast(qml.CNOT , wires=range(n_qubits), pattern="all_to_all")
        if not prod_approx:
            return qml.probs(wires=list(range(measurement_qubits)))
        else:
            return [qml.probs(i) for i in range(measurement_qubits)]

    def SU(params):
        qml.SpecialUnitary(params, wires=range(n_qubits))
        
        ZZ = qml.operation.Tensor(qml.PauliZ(0), qml.PauliZ(1))
        for i in range(2,n_qubits):
            ZZ = qml.operation.Tensor(ZZ, qml.PauliZ(i))

        return qml.expval(ZZ)
    
    def simmpleRZRY(params,cnots=True):
        qml.broadcast(qml.Hadamard, wires=range(n_qubits), pattern="single")
        qml.broadcast(qml.RZ, wires=range(n_qubits), pattern="single", parameters=params[0])
        qml.broadcast(qml.RY, wires=range(n_qubits), pattern="single", parameters=params[1])
        if cnots:
            qml.broadcast(qml.CNOT, wires=range(n_qubits), pattern="chain")

            return qml.expval(qml.PauliZ(n_qubits-1))
        else:
            ZZ = qml.operation.Tensor(qml.PauliZ(0), qml.PauliZ(1))
            for i in range(2,n_qubits):
                ZZ = qml.operation.Tensor(ZZ, qml.PauliZ(i))

            return qml.expval(ZZ)
        
    def RY(params,y=True,probs=False,prod=False, entanglement=None):
        #qml.broadcast(qml.Hadamard, wires=range(n_qubits), pattern="single")
        qml.broadcast(qml.RY, wires=range(n_qubits), pattern="single", parameters=params)
        #qml.broadcast(qml.CZ, wires=range(n_qubits), pattern="all_to_all")

        if entanglement=="all_to_all":
            qml.broadcast(qml.CNOT, wires=range(n_qubits), pattern="all_to_all")
        
        if y==True:
            #YY = qml.operation.Tensor(qml.PauliY(0), qml.PauliY(1))
            YY = [qml.PauliZ(0), qml.PauliZ(1)]
            for i in range(2,n_qubits):
                #YY = qml.operation.Tensor(YY, qml.PauliY(i))
                YY.append(qml.PauliZ(i))
            
            #return [qml.expval(i) for i in YY]
            return qml.expval(YY)

        elif probs==False:

            ZZ = qml.operation.Tensor(qml.PauliZ(0), qml.PauliZ(1))
            #ZZ = [qml.PauliZ(0), qml.PauliZ(1)]
            for i in range(2,n_qubits):
                ZZ = qml.operation.Tensor(ZZ, qml.PauliZ(i))        
                #ZZ.append(qml.PauliZ(i))        

            #return [qml.expval(i) for i in ZZ]
            return qml.expval(ZZ)

        else:
            if prod:
                return [qml.probs(i) for i in range(n_qubits)]
            else:
                return qml.probs(wires=range(n_qubits))
            
        
        
    def GHZ(params,measurement_qubits=0):
        qml.RY(params,wires=0)
        qml.broadcast(qml.CNOT, wires=range(n_qubits), pattern="chain")

        return qml.probs(wires=range(measurement_qubits))

    def random_product_state(params,gate_sequence=None):
                
        for i in range(n_qubits):
            qml.RY(np.pi / 4, wires=i)

        for ll in range(len(params)):

            for i in range(n_qubits):
                gate_sequence["{}{}".format(ll,i)](params[ll][i], wires=i)

            #for i in range(n_qubits - 1):
                #qml.CZ(wires=[i, i + 1])
    def SEL(params, measurement_qubits=0):
        qml.StronglyEntanglingLayers(params, wires=range(n_qubits))
        return qml.probs(wires=range(measurement_qubits))
    
    def RL(params, measurement_qubits=0):
        qml.RandomLayers(params, ratio_imprim=0.8 ,imprimitive=qml.CZ, wires=range(n_qubits))
        return qml.probs(wires=range(measurement_qubits))
    
    if circ == "rzry":
        qcircuit = RZRY
    elif circ == "simplified_two_design":
        qcircuit = S2D
    elif circ == "special_unitary":
        qcircuit = SU
    elif circ == "simpleRZRY":
        qcircuit = simmpleRZRY
    elif circ == "RY":
        qcircuit = RY
    elif circ == "ghz":
        qcircuit = GHZ
    elif circ == "random_product_state":
        qcircuit = random_product_state
    elif circ == "SEL":
        qcircuit = SEL
    elif circ == "RL":
        qcircuit = RL
    if not fim:
        circuit = qml.QNode(qcircuit, dev,interface="torch", diff_method="backprop")
    else:
        circuit = qml.QNode(qcircuit, dev)

    return circuit

def compute_gradient(log_prob, w):
    """Compute gradient of the log probability with respect to weights.
    
    Args:
    - log_prob (torch.Tensor): The log probability tensor.
    - w (torch.Tensor): The weights tensor, with requires_grad=True.

    Returns:
    - numpy.ndarray: The gradient of log_prob with respect to w, flattened.
    """
    if w.grad is not None:
        w.grad.zero_()
    log_prob.backward(retain_graph=True)
    
    if w.grad is None:
        raise ValueError("The gradient for the given log_prob with respect to w is None.")
    
    return w.grad.view(-1).detach().numpy()

def policy(probs, policy_type="contiguous-like", n_actions=2, n_qubits=1):

    if policy_type == "contiguous-like":
        return probs
    elif policy_type == "parity-like":
        policy = torch.zeros(n_actions)
        for i in range(len(probs)):
            a=[]
            for m in range(int(np.log2(n_actions))):
                if m==0:    
                    bitstring = np.binary_repr(i,width=n_qubits)
                else:
                    bitstring = np.binary_repr(i,width=n_qubits)[:-m]
                
                a.append(bitstring.count("1") % 2)
            policy[int("".join(str(x) for x in a),2)] += probs[i]

        return policy    
    
def compute_policy_and_gradient(args):
    n_qubits, shapes, type , n_actions, policy_type, clamp = args

    if policy_type == "parity-like":
        measure_qubits = n_qubits
    else:
        measure_qubits = int(np.log2(n_actions))

    qc = create_circuit(n_qubits, circ=type, fim=False, shots=None)

    if type == "simplified_two_design":
        weights = [np.random.uniform(-np.pi,np.pi,size=shape) for shape in shapes]    
        weights_tensor_init = torch.tensor(weights[0], requires_grad=False)
        weights_tensor_params = torch.tensor(weights[1], requires_grad=True)
        
        probs = qc(weights_tensor_init,weights_tensor_params, measurement_qubits=measure_qubits)

    else:
        weights = [np.random.uniform(-np.pi,np.pi,size=shape) for shape in shapes]    
        weights_tensor_params = torch.tensor(weights, requires_grad=True)

        probs = qc(weights_tensor_params, measurement_qubits=measure_qubits)

    pi = policy(probs, policy_type=policy_type, n_actions=n_actions, n_qubits=n_qubits)
    if clamp is not None:
        pi = torch.clamp(pi, clamp, 1)

    dist = torch.distributions.Categorical(probs=pi)
    
    action = dist.sample()
    log_prob = dist.log_prob(action)

    gradient_no_clamp = np.linalg.norm(compute_gradient(log_prob, weights_tensor_params), 2)
    return gradient_no_clamp


In [1]:
import pennylane as qml
import numpy as np
import torch
from torch.nn.parameter import Parameter
import torch.nn as nn

In [11]:
def create_zz_operator(n_qubits):
    ZZ = qml.PauliZ(0)
    for i in range(1, n_qubits):
        ZZ = qml.operation.Tensor(ZZ, qml.PauliZ(i))
    return ZZ

def measure_selection(n_qubits, measure_type, observables):
    if measure_type == 'probs':
        if observables is None:
            return qml.probs(wires=range(n_qubits))
        else:
            return qml.probs(op=observables, wires=range(n_qubits))
    elif measure_type == 'expval':
        op = observables if observables is not None else create_zz_operator(n_qubits)
        return qml.expval(op=op) 
    
def jerbi_model(n_qubits, n_layers, weight_init, measure_type, observables):

    dev = qml.device("default.qubit", wires=n_qubits)

    observables = observables if observables is not None else None
    
    weight_shapes = {"params": (n_layers + 1, n_qubits, 2)}
    init_method   = {"params": weight_init}

    @qml.qnode(dev, interface='torch')
    def qnode(inputs, params):
        qml.broadcast(qml.Hadamard, wires=range(n_qubits), pattern="single")
        for layer in range(n_layers):
            for wire in range(n_qubits):
                qml.RZ(params[layer][wire][0], wires=wire)
                qml.RY(params[layer][wire][1], wires=wire)

            qml.broadcast(qml.CNOT, wires=range(n_qubits), pattern="chain")
            for wire in range(n_qubits):
                qml.RY(inputs[wire], wires=wire)
                qml.RZ(inputs[wire], wires=wire)

        for wire in range(n_qubits):
            qml.RZ(params[-1][wire][0], wires=wire)
            qml.RY(params[-1][wire][1], wires=wire)
    
        return measure_selection(n_qubits, measure_type, observables)

    model = qml.qnn.TorchLayer(qnode, weight_shapes=weight_shapes, init_method=init_method)

    return model

In [12]:
class CircuitGenerator(nn.Module):

    def __init__(self, 
                 n_qubits, 
                 n_layers,  
                 shots, 
                 weight_init=torch.nn.init.uniform_, 
                 input_init = torch.nn.init.ones_, 
                 measure_type = 'probs', 
                 observables = None):
        super(CircuitGenerator, self).__init__()
        self.n_qubits = n_qubits                        #number of qubits
        self.n_layers = n_layers                        #number of layers
        self.shots = shots                              #number of shots
        self.measure_type = measure_type                #measure type - 'probs' or 'expval'
        self.observables = observables                  #observables if the used wants
        self.weight_init = weight_init                  #weight initialization method
        self.input_init = input_init                    #input weight initialization method



    def jerbi_model(self, input_scaling = False):
        
        if input_scaling:
            self.input_params = Parameter(torch.empty(self.n_layers, self.n_qubits, 2))
            self.input_init(self.input_params)
        else:
            self.input_params = None
            
        self.q_layers = jerbi_model(n_qubits=self.n_qubits,
                                    n_layers=self.n_layers,
                                    weight_init = self.weight_init,
                                    measure_type=self.measure_type,
                                    observables=self.observables)
        
        return self.q_layers

    def jerbi_input(self,inputs):

        if self.input_params is not None:
            inputs = inputs * self.input_params

        outputs = self.q_layers(inputs)

        return outputs

In [13]:
class PolicyType():
    
    def __init__(self, n_actions, n_qubits):
        self.n_actions = n_actions
        self.n_qubits = n_qubits

    def raw_contiguous(self,probs):

        probs_flatten = probs.flatten()
        chunk_size = len(probs_flatten) // self.n_actions
        remainder = len(probs_flatten) % self.n_actions

        # Initialize the sums list
        policy = [0] * self.n_actions

        # Iterate over each chunk
        for i in range(self.n_actions):
            # Calculate the start and end indices of the current chunk
            start = i * chunk_size
            end = (i + 1) * chunk_size

            # Add one more element to the first `remainder` chunks if needed
            if i < remainder:
                end += 1

            # Sum the elements in the current chunk
            policy[i] = sum(probs_flatten[start:end])

        return policy
    
    def raw_parity(self,probs):

        policy = torch.zeros(self.n_actions)
        for i in range(len(probs)):
            a=[]
            for m in range(int(np.log2(self.n_actions))):
                if m==0:    
                    bitstring = np.binary_repr(i,width=self.n_qubits)
                else:
                    bitstring = np.binary_repr(i,width=self.n_qubits)[:-m]
                
                a.append(bitstring.count("1") % 2)
            policy[int("".join(str(x) for x in a),2)] += probs[i]

        return policy 
    

    def softmax(self,probs,beta):



SyntaxError: invalid syntax (1322505372.py, line 29)

In [14]:
# Example usage
n_qubits = 6
n_layers = 2
shots = 1000

weight_init = torch.nn.init.uniform_
input_init = torch.nn.init.uniform_
# Initialize input parameters with ones

# Create a CircuitGenerator instance
cg = CircuitGenerator(n_qubits, n_layers, shots, weight_init, input_init)

# Get the quantum layer
quantum_layer = cg.jerbi_model(input_scaling=False)

# Generate some input data
inputs = torch.randn(n_qubits)

# Pass the input data through the quantum layer
outputs = cg.jerbi_input(inputs)
print(outputs)

tensor([8.8700e-03, 3.0239e-02, 1.2414e-02, 1.0964e-02, 7.2783e-03, 6.7797e-04,
        1.5266e-02, 5.6363e-04, 2.6536e-02, 1.6026e-02, 4.3246e-03, 7.1702e-04,
        1.7780e-02, 2.7843e-02, 6.3011e-02, 4.4059e-03, 7.1276e-04, 1.0127e-02,
        1.0341e-02, 4.7116e-03, 2.0108e-03, 1.0614e-03, 1.3933e-02, 1.8976e-06,
        1.6571e-02, 5.0250e-03, 1.3010e-04, 3.3808e-03, 1.1145e-02, 9.0339e-04,
        1.7886e-02, 9.4760e-04, 2.1516e-02, 7.1967e-02, 2.2481e-02, 2.1971e-02,
        7.7567e-03, 1.0843e-03, 1.4764e-02, 1.2350e-03, 7.4796e-02, 4.4789e-02,
        7.3209e-03, 1.4990e-03, 4.0680e-02, 7.3972e-02, 1.7529e-01, 1.0065e-02,
        4.4297e-03, 4.5065e-04, 6.3098e-03, 3.4938e-04, 5.1875e-03, 6.9385e-03,
        2.4546e-02, 1.1150e-03, 1.3268e-03, 1.9733e-04, 1.1409e-03, 1.0692e-03,
        3.1746e-03, 5.2083e-03, 1.3979e-03, 1.6574e-04],
       grad_fn=<ViewBackward0>)
